In [1]:
## Prob. 2

In [2]:
import numpy as np
import pandas as pd
import os
import sklearn
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from sklearn.linear_model import LinearRegression 

In [3]:
pwd

'/Users/pradeepsingh/cs696/hw3/hw3-pradeep'

In [4]:
fullpath = '/Users/pradeepsingh/cs696/hw3/data/multiple-sites.tsv'

In [5]:
dwell_data = pd.read_csv(fullpath, sep='\t')

In [6]:
dwell_data.head(5)

,site,dwell-time
0,13,28
1,6,33
2,17,81
3,1,67
4,16,87


### 1. Mean and Std. Dev. using Panads:

In [7]:
# compute mean of the dwell-time per site
pd_mean = dwell_data.groupby(['site']).mean()
pd_mean

,dwell-time
site,
0,79.851064
1,106.000000
2,88.229167
3,97.479167
4,94.333333
5,102.333333
6,144.192982
7,123.367347
8,94.346939


In [8]:
# compute std dev of the dwell-time per site
pd_std = dwell_data.groupby(['site']).std()
pd_std

,dwell-time
site,
0,79.135266
1,103.790126
2,88.709001
3,74.814093
4,77.725843
5,87.700618
6,146.930917
7,117.025584
8,97.564626


### 2. Mean and Std. Dev. using Spark:

In [9]:
import pyspark.sql.functions as F

In [10]:
path = "/Users/pradeepsingh/cs696/hw3/data/multiple-sites.tsv"

In [11]:
## read csv/tsv files in spark
reader = spark.read
reader.option("header",True)
reader.option("inferSchema",True)
reader.option("sep","\t")

In [12]:
df = reader.csv(path)

In [13]:
df.show()

+----+----------+
|site|dwell-time|
+----+----------+
|  13|        28|
|   6|        33|
|  17|        81|
|   1|        67|
|  16|        87|
|   2|        56|
|   2|        72|
|   6|       297|
|  13|         2|
|   1|        94|
|   5|       210|
|   4|        49|
|   8|       152|
|   2|        31|
|   5|       146|
|  10|       226|
|   9|       152|
|   1|         7|
|  12|        17|
|  10|       105|
+----+----------+
only showing top 20 rows



In [14]:
df.printSchema()

root
 |-- site: integer (nullable = true)
 |-- dwell-time: integer (nullable = true)



In [15]:
# Group dwell-time by sites 
# Compute mean and std. dev of dwell-time per site

dfGrouped = df.groupBy("site").agg(F.mean("dwell-time").alias("Mean of Dwell-time/site"),
                            F.stddev("dwell-time").alias("Std dev of Dwell-time/site"))

In [16]:
dfGrouped.sort("site").show()

+----+-----------------------+--------------------------+
|site|Mean of Dwell-time/site|Std dev of Dwell-time/site|
+----+-----------------------+--------------------------+
|   0|      79.85106382978724|         79.13526634363751|
|   1|                  106.0|        103.79012594607659|
|   2|      88.22916666666667|         88.70900061876551|
|   3|      97.47916666666667|          74.8140928763278|
|   4|      94.33333333333333|         77.72584297816697|
|   5|     102.33333333333333|          87.7006178654617|
|   6|     144.19298245614036|        146.93091654103628|
|   7|     123.36734693877551|        117.02558371953526|
|   8|      94.34693877551021|         97.56462623572646|
|   9|                  89.82|        100.47046271193848|
|  10|     129.95238095238096|         90.10940569215181|
|  11|      96.98214285714286|         78.94635487501591|
|  12|      80.95081967213115|         68.98729985282513|
|  13|      90.73770491803279|        102.01387187360751|
|  14|      74